In [1]:
import pandas as pd

In [2]:
movies=pd.read_csv('movies.csv')
links=pd.read_csv('links.csv')

In [3]:
movies=movies.merge(links,on='movieId')

In [4]:
movies.head(1)

movieId             title                                       genres  \
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   

   imdbId  tmdbId  
0  114709   862.0

In [5]:
movies=movies.iloc[:25000,:]

In [6]:
movies.head(11)

movieId                               title  \
0         1                    Toy Story (1995)   
1         2                      Jumanji (1995)   
2         3             Grumpier Old Men (1995)   
3         4            Waiting to Exhale (1995)   
4         5  Father of the Bride Part II (1995)   
5         6                         Heat (1995)   
6         7                      Sabrina (1995)   
7         8                 Tom and Huck (1995)   
8         9                 Sudden Death (1995)   
9        10                    GoldenEye (1995)   
10       11      American President, The (1995)   

                                         genres  imdbId   tmdbId  
0   Adventure|Animation|Children|Comedy|Fantasy  114709    862.0  
1                    Adventure|Children|Fantasy  113497   8844.0  
2                                Comedy|Romance  113228  15602.0  
3                          Comedy|Drama|Romance  114885  31357.0  
4                                        Comedy  113041  11862.0  
5                         Action|Crime|Thriller  113277    949.0  
6                                Comedy|Romance  114319  11860.0  
7                            Adventure|Children  112302  45325.0  
8                                        Action  114576   9091.0  
9                     Action|Adventure|Thriller  113189    710.0  
10                         Comedy|Drama|Romance  112346   9087.0

In [7]:
def title(value):
    value=value.replace(" ","")
    c=value[-2:-6:-1]
    if(c.isdigit()):
        return int(c[::-1])
    else:
        return None
movies['year']=movies['title'].apply(title)

In [8]:
movies.dtypes

movieId      int64
title       object
genres      object
imdbId       int64
tmdbId     float64
year       float64
dtype: object

In [9]:
movies['year'].value_counts()

2009.0    945
2008.0    879
2011.0    854
2012.0    838
2010.0    821
         ... 
1894.0      1
1896.0      1
1903.0      1
1899.0      1
1900.0      1
Name: year, Length: 115, dtype: int64

In [10]:
movies.isnull().sum()

movieId      0
title        0
genres       0
imdbId       0
tmdbId     100
year         2
dtype: int64

In [11]:
movies.dropna(inplace=True)

In [12]:
movies.isnull().sum()

movieId    0
title      0
genres     0
imdbId     0
tmdbId     0
year       0
dtype: int64

In [13]:
movies['year']=movies['year'].astype('int64')

In [14]:
movies['year'].min()

1894

In [15]:
movies['year'].max()

2015

In [16]:
movies['tmdbId']=movies['tmdbId'].astype('int64')

In [17]:
movies.dtypes

movieId     int64
title      object
genres     object
imdbId      int64
tmdbId      int64
year        int64
dtype: object

In [18]:
def genres(i):
    l=[]
    i=i.replace('|',',')
    l.extend(i.split(','))
    return l
movies['genres']=movies['genres'].apply(genres)

In [19]:
movies['genres']=movies['genres'].apply(lambda x:" ".join(x))

In [20]:
movies['genres']

0        Adventure Animation Children Comedy Fantasy
1                         Adventure Children Fantasy
2                                     Comedy Romance
3                               Comedy Drama Romance
4                                             Comedy
                            ...                     
24995                                         Comedy
24996                 Children Comedy Fantasy Sci-Fi
24997                             Adventure Children
24998                                         Horror
24999                                Horror Thriller
Name: genres, Length: 24898, dtype: object

In [21]:
from sklearn.feature_extraction.text import CountVectorizer 

In [22]:
cv=CountVectorizer()

In [23]:
vectors=cv.fit_transform(movies['genres']).toarray()

In [24]:
cv.get_feature_names()

c:\users\bansa\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'fi',
 'film',
 'genres',
 'horror',
 'imax',
 'listed',
 'musical',
 'mystery',
 'no',
 'noir',
 'romance',
 'sci',
 'thriller',
 'war',
 'western']

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
similar_movies=cosine_similarity(vectors)

In [26]:
def title2(value):
    aa=value.split()
    bb=aa[:len(aa)-1]
    return " ".join(bb)
movies['title']=movies['title'].apply(title2)

In [27]:
movies.loc[10,'title']='The American President'

In [28]:
movies['oldtitle']=movies['title']

In [29]:
movies['oldtitle']=movies['oldtitle'].apply(lambda x:x.lower())
movies['oldtitle']=movies['oldtitle'].apply(lambda x:x.replace(" ",""))

In [85]:
def recommand(movie):
    movie=movie.lower().replace(" ","")
    if movie not in movies['oldtitle'].values:
        return "No Movie found,please try again"
    else:
        movie_index=movies[movies['oldtitle']==movie].index[0]
        distances=enumerate(list(similar_movies[movie_index]))
        j=sorted(distances,reverse=True,key=lambda x:x[1])
        movie_rec=j[:8]
        l=[]
        for i in movie_rec:
            index,distance=i
            l.append(movies.loc[index,'title'])
    return l
recommand("true crime")

['Seven (a.k.a. Se7en)',
 'Just Cause',
 'Underneath',
 'True Crime',
 'American in Paris, An',
 'Fire on the Mountain',
 'Merry War, A',
 'Suspicion']

In [31]:
movies.head(10)

movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   
5        6                         Heat   
6        7                      Sabrina   
7        8                 Tom and Huck   
8        9                 Sudden Death   
9       10                    GoldenEye   

                                        genres  imdbId  tmdbId  year  \
0  Adventure Animation Children Comedy Fantasy  114709     862  1995   
1                   Adventure Children Fantasy  113497    8844  1995   
2                               Comedy Romance  113228   15602  1995   
3                         Comedy Drama Romance  114885   31357  1995   
4                                       Comedy  113041   11862  1995   
5                        Action Crime Thriller  113277     949  1995   
6                               Comedy Romance  114319   11860  1995   
7                           Adventure Children  112302   45325  1995   
8                                       Action  114576    9091  1995   
9                    Action Adventure Thriller  113189     710  1995   

                 oldtitle  
0                toystory  
1                 jumanji  
2          grumpieroldmen  
3         waitingtoexhale  
4  fatherofthebridepartii  
5                    heat  
6                 sabrina  
7              tomandhuck  
8             suddendeath  
9               goldeneye

In [32]:
# import pickle

In [33]:
# pickle.dump(movies.to_dict(),open("movies2.pkl","wb"))

In [34]:
# pickle.dump(similar_movies2,open("similar.pkl","wb"))

In [35]:
# similar_movies2=similar_movies[:10000]


In [36]:
# similar_movies3=similar_movies[10000:]
# pickle.dump(similar_movies3,open("similar2.pkl","wb"))

In [37]:
movies.to_dict()

{'movieId': {0: 1,
  1: 2,
  2: 3,
  3: 4,
  4: 5,
  5: 6,
  6: 7,
  7: 8,
  8: 9,
  9: 10,
  10: 11,
  11: 12,
  12: 13,
  13: 14,
  14: 15,
  15: 16,
  16: 17,
  17: 18,
  18: 19,
  19: 20,
  20: 21,
  21: 22,
  22: 23,
  23: 24,
  24: 25,
  25: 26,
  26: 27,
  27: 28,
  28: 29,
  29: 30,
  30: 31,
  31: 32,
  32: 33,
  33: 34,
  34: 35,
  35: 36,
  36: 37,
  37: 38,
  38: 39,
  39: 40,
  40: 41,
  41: 42,
  42: 43,
  43: 44,
  44: 45,
  45: 46,
  46: 47,
  47: 48,
  48: 49,
  49: 50,
  50: 51,
  51: 52,
  52: 53,
  53: 54,
  54: 55,
  55: 56,
  56: 57,
  57: 58,
  58: 59,
  59: 60,
  60: 61,
  61: 62,
  62: 63,
  63: 64,
  64: 65,
  65: 66,
  66: 67,
  67: 68,
  68: 69,
  69: 70,
  70: 71,
  71: 72,
  72: 73,
  73: 74,
  74: 75,
  75: 76,
  76: 77,
  77: 78,
  78: 79,
  79: 80,
  80: 81,
  81: 82,
  82: 83,
  83: 84,
  84: 85,
  85: 86,
  86: 87,
  87: 88,
  88: 89,
  89: 90,
  90: 92,
  91: 93,
  92: 94,
  93: 95,
  94: 96,
  95: 97,
  96: 98,
  97: 99,
  98: 100,
  99: 101,
  100:

In [60]:
def listgenre(value):
    value=value.lower().split()
    return value
movies['genres']=movies['genres'].apply(listgenre)

In [68]:
movies90s=movies.loc[(movies['year']>1990) & (movies['year']<2000)]

In [69]:
movies80s=movies.loc[(movies['year']>1980) & (movies['year']<1990)]

In [70]:
movies2000s=movies.loc[(movies['year']>2000) & (movies['year']<2010)]

In [73]:
latest=movies.loc[(movies['year']>2010) & (movies['year']<2020)]

In [76]:
import pickle

In [77]:
pickle.dump(movies90s.to_dict(),open("movies90s.pkl","wb"))

In [78]:
pickle.dump(movies80s.to_dict(),open("movies80s.pkl","wb"))

In [79]:
pickle.dump(movies2000s.to_dict(),open("movies2000s.pkl","wb"))

In [80]:
pickle.dump(latest.to_dict(),open("latest.pkl","wb"))

In [91]:
import pickle
import pandas as pd

a90s=pickle.load(open("movies90s.pkl","rb"))
a80s=pickle.load(open("movies80s.pkl","rb"))
a2000s=pickle.load(open("movies2000s.pkl","rb"))
alatest=pickle.load(open("latest.pkl","rb"))

a90s=pd.DataFrame(a90s)
a80s=pd.DataFrame(a80s)
a80s=a80s.reset_index()

a2000s=pd.DataFrame(a2000s)
alatest=pd.DataFrame(alatest)

a80s

index  movieId                             title  \
0       140      142                  Shadows (Cienie)   
1       536      541                      Blade Runner   
2       584      592                            Batman   
3       602      610                       Heavy Metal   
4       668      681     Coup de torchon (Clean Slate)   
...     ...      ...                               ...   
2210  24851   122451                        Dark Tower   
2211  24896   122553        Assault of the Party Nerds   
2212  24930   122623                      Oliver Twist   
2213  24940   122647                  Movers & Shakers   
2214  24981   122731  National Lampoon's Movie Madness   

                                              genres  imdbId  tmdbId  year  \
0                                            [drama]   94878  525153  1988   
1                         [action, sci-fi, thriller]   83658      78  1982   
2                          [action, crime, thriller]   96895     268  1989   
3     [action, adventure, animation, horror, sci-fi]   82509   11827  1981   
4                                            [crime]   82206   35797  1981   
...                                              ...     ...     ...   ...   
2210                              [horror, thriller]   92831  106280  1989   
2211                                        [comedy]   96839   30027  1989   
2212                                  [crime, drama]   84438   49483  1982   
2213                                        [comedy]   89628   89743  1985   
2214                                        [comedy]   82799   33782  1983   

                           oldtitle  
0                   shadows(cienie)  
1                       bladerunner  
2                            batman  
3                        heavymetal  
4         coupdetorchon(cleanslate)  
...                             ...  
2210                      darktower  
2211         assaultofthepartynerds  
2212                    olivertwist  
2213                 movers&shakers  
2214  nationallampoon'smoviemadness  

[2215 rows x 8 columns]

In [102]:
from numpy import random
def a80ss():
    idxlist=[]
    suggest=[]
    suggest_poster=[]
    released_date=[]
    overview=[]
    for num in range(0,20):
        idxlist.append(random.randint(2100))
    for j in idxlist:
        index_id=a80s.loc[j,'tmdbId']
        print(j,index_id)
        suggest.append(a80s.loc[j,'title'])
    return suggest

In [103]:
a80ss()

2052 41943
494 44018
1262 78041
1277 8408
424 25010
1901 262475
522 10117
1335 42094
1371 27171
767 11966
1040 28893
1831 31300
751 38617
1303 62327


['Plot Against Harry, The',
 'Vagabond (Sans toit ni loi)',
 'Big Trouble',
 'Day of the Dead',
 'Flamingo Kid, The',
 'G.I. Joe: The Revenge of Cobra',
 'Action Jackson',
 'Micki + Maude',
 'Lonely Guy, The',
 'Short Circuit 2',
 'Galaxy of Terror (Quest)',
 'Under the Sun of Satan (Sous le soleil de Satan)',
 'Off Limits',
 'Promised Land']

In [105]:
a80s.isnull().sum()

index       0
movieId     0
title       0
genres      0
imdbId      0
tmdbId      0
year        0
oldtitle    0
dtype: int64